In [1]:
import datetime
import duckdb
import json
import os
import pandas as pd
import requests
import time
import tqdm
from typing import Dict, List

In [2]:
SLEEPER_REST_API = "https://api.sleeper.com"
SLEEPER_GRAPHQL_API = "https://sleeper.com/graphql"
# ALLOWED_POSITIONS = set(["RB", "FB", "WR", "TE"])
ALLOWED_POSITIONS = set(["RB", "FB"])


def get_player_reports(player_id: str, limit: int) -> List[Dict]:
    operation_name = "get_player_news"
    kwargs = {
        "operation_name": operation_name,
        "player_id": player_id,
        "limit": limit,
    }
    query = """
        query {operation_name} {{
            get_player_news(sport: "nfl", player_id: "{player_id}", limit: {limit}){{
                metadata
                player_id
                published
                source
                source_key
                sport
            }}
        }}
    """.format(**kwargs)
    body = {
        "operationName": operation_name,
        "query": query,
        "variables": {},
    }
    time.sleep(0.1)
    req = requests.post(SLEEPER_GRAPHQL_API, json=body)
    res = req.json()
    reports = res.get("data", {}).get(operation_name, [])
    return reports


def transform_reports(raw_reports: List[Dict]) -> List[Dict]:
    reports = [
        {
            "report_id": r.get("source") + "_" + r.get("source_key"),
            "player_id": str(r.get("player_id")),
            "source": r.get("source"),
            "category": "retrospective" if r.get("metadata", {}).get("analysis") is not None else "prospective",
            "published_at": r.get("published"),
            "title": r.get("metadata", {}).get("title"),
            "description": r.get("metadata", {}).get("description"),
            "analysis": r.get("metadata", {}).get("analysis"),
        }
        for r in raw_reports
    ]
    output = list(filter(lambda r: r.get("category") == "prospective", reports))
    return output


def get_games_for_week(season: int, week: int) -> List[Dict]:
    operation_name = "scores"
    kwargs = {
        "operation_name": operation_name,
        "season": season,
        "week": week,
    }
    query = """
        query {operation_name} {{
            scores(sport: "nfl", season_type: "regular", season: "{season}", week: {week}){{
              date
              game_id
              metadata
              season
              season_type
              sport
              status
              week
              start_time
            }}
        }}
    """.format(**kwargs)
    body = {
        "operationName": operation_name,
        "query": query,
        "variables": {},
    }
    time.sleep(0.1)
    req = requests.post(SLEEPER_GRAPHQL_API, json=body)
    res = req.json()
    games = res.get("data", {}).get(operation_name, [])
    return games


def get_games_for_season(season: int, max_weeks: int) -> List[Dict]:
    output = []
    for week in range(1, max_weeks + 1):
        games = get_games_for_week(season, week)
        output.extend(games)
    return output


def transform_games(raw_games: List[Dict]) -> List[Dict]:
    return [
        {
            "game_id": r.get("game_id"),
            "season": r.get("season"),
            "week": r.get("week"),
            "date": r.get("date"),
            "started_at": r.get("start_time"),
            "home_team": r.get("metadata", {}).get("home_team"),
            "away_team": r.get("metadata", {}).get("away_team"),
            "home_score": int(r.get("metadata", {}).get("home_score", 0)),
            "away_score": int(r.get("metadata", {}).get("away_score", 0)),
        }
        for r in raw_games
    ]


def get_all_players() -> List[Dict]:
    url = f"{SLEEPER_REST_API}/v1/players/nfl"
    req = requests.get(url)
    players_by_id = req.json()
    players = [
        {
            "player_id": str(p.get("player_id")),
            "name": p.get("full_name"),
            "position": p.get("position"),
            "current_team": p.get("team"),
            "jersey_number": int(p.get("number")) if p.get("number") is not None else None,
        }
        for p in players_by_id.values()
    ]
    filtered_players = list(filter(lambda p: p.get("position") in ALLOWED_POSITIONS, players))
    return filtered_players


def get_player_week_stats(player_id: str, season: int) -> List[Dict]:
    url = f"{SLEEPER_REST_API}/stats/nfl/player/{player_id}?season_type=regular&season={season}&grouping=week"
    time.sleep(0.1)
    req = requests.get(url)
    res = req.json()
    raw = res.values()
    week_stats = list(filter(lambda v: v is not None, raw))
    return week_stats


def transform_week_stats(raw_week_stats: List[Dict]) -> List[Dict]:
    return [
        {
            "stat_id": r.get("season") + "_" + str(r.get("week")) + "_" + r.get("player_id"),
            "week_id": r.get("season") + "_" + str(r.get("week")),
            "player_id": str(r.get("player_id")),
            "game_id": r.get("game_id"),
            "season": r.get("season"),
            "week": r.get("week"),
            "team": r.get("team"),
            "opponent": r.get("opponent"),
            "played_at": int(datetime.datetime.strptime(r.get("date"), "%Y-%m-%d").timestamp() * 1000),
            "was_active": int(r.get("stats", {}).get("gms_active", 0)) > 0,
            "was_played": int(r.get("stats", {}).get("gp", 0)) > 0,
            "offensive_snaps": int(r.get("stats", {}).get("off_snp", 0)),
            "rushing_attempts": int(r.get("stats", {}).get("rush_att", 0)),
            "rushing_yards": int(r.get("stats", {}).get("rush_yd", 0)),
            "rushing_touchdowns": int(r.get("stats", {}).get("rush_td", 0)),
            "receiving_targets": int(r.get("stats", {}).get("rec_tgt", 0)),
            "receiving_catches": int(r.get("stats", {}).get("rec", 0)),
            "receiving_yards": int(r.get("stats", {}).get("rec_yd", 0)),
            "receiving_touchdowns": int(r.get("stats", {}).get("rec_td", 0)),
            "standard_scoring_fantasy_points": int(r.get("stats", {}).get("pts_std", 0)),
            "standard_scoring_position_rank": int(r.get("stats", {}).get("pos_rank_std", 0)),
            "ppr_scoring_fantasy_points": int(r.get("stats", {}).get("pts_ppr", 0)),
            "ppr_scoring_position_rank": int(r.get("stats", {}).get("pos_rank_ppr", 0)),
        }
        for r in raw_week_stats
    ]

In [3]:
DROP_IF_EXISTS_QUERY = """
DROP TABLE IF EXISTS {table_name}
"""

CREATE_FROM_CSV_QUERY = """
CREATE TABLE {table_name} AS
SELECT *
FROM read_csv_auto('{filename}')
""".strip()

CREATE_TEMPLATE_BY_FORMAT = {
    "csv": CREATE_FROM_CSV_QUERY,
}


def create_table(con, config: Dict):
    table_name = config.get("name")
    if not table_name:
        raise ValueError("Must specify table name.")

    table_path = config.get("path")
    if not table_path:
        raise ValueError("Must specify table path.")

    table_format = config.get("format")
    if not table_format:
        raise ValueError("Must specify table format.")
    if table_format not in CREATE_TEMPLATE_BY_FORMAT:
        valid_formats = ", ".join(CREATE_TEMPLATE_BY_FORMAT.keys())
        error = f"Invalid format `{table_format}`. Must be one of: {valid_formats}."
        raise ValueError(error)

    drop_query = DROP_IF_EXISTS_QUERY.format(table_name=table_name)
    con.execute(drop_query)

    template = CREATE_TEMPLATE_BY_FORMAT.get(table_format)
    create_query = template.format(table_name=table_name, filename=table_path)
    con.execute(create_query)


def create_database(database_name: str):
    if os.path.exists(database_name):
        os.remove(database_name)


def create_tables(database_name: str, table_configs: List[Dict]):
    with duckdb.connect(database_name) as con:
        for table_config in table_configs:
            create_table(con, table_config)

In [4]:
DATABASE = "../database.db"

In [5]:
create_database(database_name=DATABASE)

In [6]:
SEASON = 2025
WEEKS = 18
MAX_REPORTS_PER_WEEK = 15
MAX_REPORTS = WEEKS * MAX_REPORTS_PER_WEEK

In [7]:
raw_games = get_games_for_season(season=SEASON, max_weeks=WEEKS)
games = transform_games(raw_games)
df_games = pd.DataFrame(games)
df_games.to_csv("../data/processed/game.csv", index=False)

del raw_games
del games
del df_games

In [8]:
df_players = pd.DataFrame(get_all_players())
df_players.player_id = df_players.player_id.astype(str)
df_players.jersey_number = df_players.jersey_number.astype("Int64")
df_players.to_csv("../data/processed/player.csv", index=False)

del df_players

In [9]:
create_tables(
    database_name=DATABASE,
    table_configs=[
        {
            "name": "player",
            "path": "../data/processed/player.csv",
            "format": "csv",
        },
        {
            "name": "game",
            "path": "../data/processed/game.csv",
            "format": "csv",
        },
    ]
)

In [10]:
target_player_ids = []
with duckdb.connect(DATABASE) as con:
    cur = con.sql("""
    SELECT DISTINCT player_id
    FROM player p
    WHERE
        current_team IS NOT NULL
        AND position IN ('QB', 'RB', 'FB', 'WR', 'TE', 'K')
    ;
    """)
    distinct_player_ids = cur.fetchall()
    target_player_ids = [pid for (pid, ) in distinct_player_ids]

In [11]:
raw_stats = []
for player_id in tqdm.tqdm(target_player_ids):
    player_stats = get_player_week_stats(player_id=player_id, season=SEASON)
    raw_stats.extend(player_stats)

stats = transform_week_stats(raw_stats)
df_stats = pd.DataFrame(stats)
df_stats.player_id = df_stats.player_id.astype(str)
df_stats.to_csv("../data/processed/player_game_outcome.csv", index=False)

del raw_stats
del stats
del df_stats

100%|█████████████████████████████████████████| 211/211 [00:53<00:00,  3.96it/s]


In [12]:
create_tables(
    database_name=DATABASE,
    table_configs=[
        {
            "name": "player_game_outcome",
            "path": "../data/processed/player_game_outcome.csv",
            "format": "csv",
        },
    ]
)

In [13]:
with duckdb.connect(DATABASE) as con:
    con.execute("""
    CREATE OR REPLACE TABLE player_season_main_team AS
    WITH
    player_season_team_count AS (
        SELECT
            player_id,
            season,
            team,
            count(1) AS weeks,
        FROM player_game_outcome
        GROUP BY
            player_id,
            season,
            team
    ),
    player_season_main_team AS (
        SELECT
            player_id,
            season,
            arg_max(team, weeks) AS main_team,
            count(1) AS unique_teams,
        FROM player_season_team_count
        GROUP BY
            player_id,
            season
        HAVING unique_teams == 1
    )
    SELECT
        player_id,
        season,
        main_team,
    FROM player_season_main_team
    ;
    """)

    con.sql("""
    SELECT *
    FROM player_season_main_team
    LIMIT 5
    ;
    """).show()

┌───────────┬────────┬───────────┐
│ player_id │ season │ main_team │
│   int64   │ int64  │  varchar  │
├───────────┼────────┼───────────┤
│     12543 │   2025 │ CIN       │
│      8151 │   2025 │ SEA       │
│      9508 │   2025 │ TEN       │
│     11655 │   2025 │ NYG       │
│     12527 │   2025 │ LV        │
└───────────┴────────┴───────────┘



In [14]:
with duckdb.connect(DATABASE) as con:
    con.execute("""
    CREATE OR REPLACE TABLE player_game_assignment AS
    SELECT
        t.player_id,
        t.season,
        g.week,
        g.game_id,
        g.started_at,
        t.main_team AS team,
        CASE
            WHEN t.main_team = g.home_team THEN g.away_team
            WHEN t.main_team = g.away_team THEN g.home_team
            ELSE NULL
        END AS opponent,
    FROM player_season_main_team t
    LEFT JOIN game g
        ON (t.main_team = g.home_team OR t.main_team = g.away_team)
        AND t.season = g.season
    ;
    """)

    con.sql("""
    SELECT
        *,
        strftime(to_timestamp(started_at / 1000), '%Y-%m-%d %I:%M %p') AS started_at,
    FROM player_game_assignment
    ORDER BY
        season ASC,
        player_id ASC,
        week ASC
    LIMIT 17
    ;
    """).show()

┌───────────┬────────┬───────┬───────────┬───────────────┬─────────┬──────────┬─────────────────────┐
│ player_id │ season │ week  │  game_id  │  started_at   │  team   │ opponent │     started_at      │
│   int64   │ int64  │ int64 │   int64   │     int64     │ varchar │ varchar  │       varchar       │
├───────────┼────────┼───────┼───────────┼───────────────┼─────────┼──────────┼─────────────────────┤
│      1379 │   2025 │     1 │ 202510130 │ 1757275500000 │ SF      │ SEA      │ 2025-09-07 03:05 PM │
│      1379 │   2025 │     2 │ 202510222 │ 1757869200000 │ SF      │ NO       │ 2025-09-14 12:00 PM │
│      1379 │   2025 │     3 │ 202510331 │ 1758486300000 │ SF      │ ARI      │ 2025-09-21 03:25 PM │
│      1379 │   2025 │     4 │ 202510431 │ 1759089900000 │ SF      │ JAX      │ 2025-09-28 03:05 PM │
│      1379 │   2025 │     5 │ 202510532 │ 1759450500000 │ SF      │ LAR      │ 2025-10-02 07:15 PM │
│      1379 │   2025 │     6 │ 202510633 │ 1760300700000 │ SF      │ TB       │ 20

In [15]:
report_player_ids = []
with duckdb.connect(DATABASE) as con:
    cur = con.sql("""
    SELECT DISTINCT player_id
    FROM player_season_main_team
    ;
    """)
    single_team_player_ids = cur.fetchall()
    report_player_ids = [pid for (pid, ) in single_team_player_ids]

In [16]:
raw_reports = []
for player_id in tqdm.tqdm(report_player_ids):
    player_reports = get_player_reports(player_id=player_id, limit=MAX_REPORTS)
    raw_reports.extend(player_reports)

reports = transform_reports(raw_reports)
df_reports = pd.DataFrame(reports)
df_reports.player_id = df_reports.player_id.astype(str)
df_reports.to_csv("../data/processed/report.csv", index=False)

del raw_reports
del reports
del df_reports

100%|█████████████████████████████████████████| 178/178 [00:44<00:00,  3.99it/s]


In [17]:
create_tables(
    database_name=DATABASE,
    table_configs=[
        {
            "name": "report",
            "path": "../data/processed/report.csv",
            "format": "csv",
        },
    ]
)

In [18]:
with duckdb.connect(DATABASE) as con:
    con.execute("""
    CREATE OR REPLACE TABLE report_game_assignment AS
    WITH
    report_upcoming_game_closest AS (
        SELECT
            r.report_id,
            arg_min(g.game_id, g.started_at) AS closest_game_id
        FROM report r
        LEFT JOIN player_game_assignment g
            ON r.player_id = g.player_id
        WHERE
            g.started_at > r.published_at
        GROUP BY r.report_id
    ),
    report_previous_game_closest AS (
        SELECT
            r.report_id,
            arg_max(g.game_id, g.started_at) AS closest_game_id
        FROM report r
        LEFT JOIN player_game_assignment g
            ON r.player_id = g.player_id
        WHERE
            r.published_at > g.started_at
        GROUP BY r.report_id
    ),
    report_game_assignment AS (
        SELECT
            r.report_id,
            u.closest_game_id AS upcoming_game_id,
            v.closest_game_id AS previous_game_id,
        FROM report r
        LEFT JOIN report_upcoming_game_closest u
            ON r.report_id = u.report_id
        LEFT JOIN report_previous_game_closest v
            ON r.report_id = v.report_id
    )
    SELECT *
    FROM report_game_assignment
    ;
    """)

    con.sql("""
    SELECT *
    FROM report_game_assignment
    LIMIT 10
    ;
    """).show()

┌───────────────────┬──────────────────┬──────────────────┐
│     report_id     │ upcoming_game_id │ previous_game_id │
│      varchar      │      int64       │      int64       │
├───────────────────┼──────────────────┼──────────────────┤
│ rotoballer_212302 │        202511823 │        202511725 │
│ rotoballer_212282 │        202511823 │        202511725 │
│ rotoballer_212122 │        202511725 │        202511623 │
│ rotoballer_212063 │        202511725 │        202511623 │
│ rotoballer_211483 │        202511623 │        202511523 │
│ rotoballer_211343 │        202511623 │        202511523 │
│ rotoballer_211064 │        202511623 │        202511523 │
│ rotoballer_210361 │        202511523 │        202511321 │
│ rotoballer_210278 │        202511523 │        202511321 │
│ rotoballer_209731 │        202511523 │        202511321 │
├───────────────────┴──────────────────┴──────────────────┤
│ 10 rows                                       3 columns │
└───────────────────────────────────────

In [19]:
with duckdb.connect(DATABASE) as con:
    con.execute("""
    CREATE OR REPLACE TABLE comparable_report AS
    SELECT
        r.report_id
    FROM report r
    LEFT JOIN report_game_assignment a
        ON r.report_id = a.report_id
    LEFT JOIN player_game_assignment uga
        ON a.upcoming_game_id = uga.game_id
        AND r.player_id = uga.player_id
    LEFT JOIN player_game_assignment vga
        ON a.previous_game_id = vga.game_id
        AND r.player_id = vga.player_id
    LEFT JOIN player_game_outcome ugo
        ON uga.game_id = ugo.game_id
        AND uga.player_id = ugo.player_id
    LEFT JOIN player_game_outcome vgo
        ON vga.game_id = vgo.game_id
        AND vga.player_id = vgo.player_id
    WHERE
        uga.week > 1
        AND ugo.player_id IS NOT NULL
        AND vgo.player_id IS NOT NULL
    ;
    """)

    con.sql("""
    SELECT
        *
    FROM comparable_report
    LIMIT 5
    ;
    """).show()

┌───────────────────┐
│     report_id     │
│      varchar      │
├───────────────────┤
│ rotoballer_212122 │
│ rotoballer_212063 │
│ rotoballer_211483 │
│ rotoballer_211343 │
│ rotoballer_211064 │
└───────────────────┘



In [20]:
with duckdb.connect(DATABASE) as con:
    con.sql("""
    SELECT *
    FROM player
    WHERE current_team IS NOT NULL
    LIMIT 5
    ;
    """).show()

    
    con.sql("""
    SELECT *
    FROM game
    LIMIT 5
    ;
    """).show()

    con.sql("""
    SELECT
        player_id,
        season,
        week,
        team,
        opponent,
        played_at,
        rushing_attempts,
        rushing_yards,
    FROM player_game_outcome
    LIMIT 5
    ;
    """).show()

    con.sql("""
    SELECT
        report_id,
        player_id,
        published_at,
        title,
    FROM report
    LIMIT 5
    ;
    """).show()

┌───────────┬─────────────────┬──────────┬──────────────┬───────────────┐
│ player_id │      name       │ position │ current_team │ jersey_number │
│   int64   │     varchar     │ varchar  │   varchar    │     int64     │
├───────────┼─────────────────┼──────────┼──────────────┼───────────────┤
│      1408 │ Le'Veon Bell    │ RB       │ TB           │             6 │
│      6918 │ Salvon Ahmed    │ RB       │ IND          │            36 │
│     11584 │ Bucky Irving    │ RB       │ TB           │             7 │
│     11649 │ Kendall Milton  │ RB       │ CIN          │            36 │
│     12507 │ Omarion Hampton │ RB       │ LAC          │             8 │
└───────────┴─────────────────┴──────────┴──────────────┴───────────────┘

┌───────────┬────────┬───────┬────────────┬───────────────┬───────────┬───────────┬────────────┬────────────┐
│  game_id  │ season │ week  │    date    │  started_at   │ home_team │ away_team │ home_score │ away_score │
│   int64   │ int64  │ int64 │    date 

In [21]:
table_names = [
    "player",
    "player_season_main_team",
    "game",
    "player_game_assignment",
    "player_game_outcome",
    "report",
    "report_game_assignment",
    "comparable_report",
]
table_counts = []
with duckdb.connect(DATABASE) as con:
    for table_name in table_names:
        query = """
        SELECT count(1)
        FROM {table_name}
        ;
        """.format(table_name=table_name)
        cur = con.sql(query)
        count = cur.fetchall()[0][0]
        table_counts.append({
            "table": table_name,
            "rows": count,
        })

pd.DataFrame(table_counts)

,table,rows
0,player,975
1,player_season_main_team,178
2,game,272
3,player_game_assignment,3026
4,player_game_outcome,2704
5,report,5224
6,report_game_assignment,5224
7,comparable_report,2040


In [22]:
with duckdb.connect(DATABASE) as con:
    con.sql("""
    SELECT count(DISTINCT player_id) AS players_all
    FROM player
    ;
    """).show()

    con.sql("""
    SELECT count(DISTINCT player_id) AS players_current
    FROM player
    WHERE
        current_team IS NOT NULL
        AND position IN ('QB', 'RB', 'FB', 'WR', 'TE', 'K')
    ;
    """).show()

    con.sql("""
    SELECT count(DISTINCT player_id) AS players_with_stats
    FROM player_game_outcome
    ;
    """).show()

    con.sql("""
    SELECT count(DISTINCT player_id) AS players_on_single_team
    FROM player_season_main_team
    ;
    """).show()

    con.sql("""
    SELECT count(DISTINCT player_id) AS players_with_reports
    FROM report
    ;
    """).show()

    con.sql("""
    SELECT count(DISTINCT r.player_id) AS players_with_reports_before_games
    FROM report_game_assignment rga
    LEFT JOIN report r
        ON rga.report_id = r.report_id
    ;
    """).show()

    con.sql("""
    SELECT count(DISTINCT r.player_id) AS players_with_comparable_reports
    FROM comparable_report c
    LEFT JOIN report r
        ON c.report_id = r.report_id
    ;
    """).show()

┌─────────────┐
│ players_all │
│    int64    │
├─────────────┤
│         975 │
└─────────────┘

┌─────────────────┐
│ players_current │
│      int64      │
├─────────────────┤
│             211 │
└─────────────────┘

┌────────────────────┐
│ players_with_stats │
│       int64        │
├────────────────────┤
│                196 │
└────────────────────┘

┌────────────────────────┐
│ players_on_single_team │
│         int64          │
├────────────────────────┤
│                    178 │
└────────────────────────┘

┌──────────────────────┐
│ players_with_reports │
│        int64         │
├──────────────────────┤
│                  134 │
└──────────────────────┘

┌───────────────────────────────────┐
│ players_with_reports_before_games │
│               int64               │
├───────────────────────────────────┤
│                               134 │
└───────────────────────────────────┘

┌─────────────────────────────────┐
│ players_with_comparable_reports │
│              int64        